<a href="https://colab.research.google.com/github/Bruno-GSilva/price-prediction/blob/main/model_2/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 2

### Mount Drive to access data

In [181]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.functional import F
import torch.optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data Loader

In [183]:
class PriceDataset(Dataset):

  def __init__(self, transform=None):

    #Read CSV file
    df = pd.read_csv("/content/drive/MyDrive/data/PETR4.SA.csv", parse_dates=[0], infer_datetime_format=True)
    df.dropna()

    #Add collumn indicating when there is a time skip > 1 day
    time_skip = [ 0 if index == 0 else 1 if (row['Date'] - df['Date'][index - 1]).days > 1 else 0 for index, row in df.iterrows() ]
    df['Skip dates'] = time_skip

    #Filter usefull collumns
    df_filter = df.filter(['Open', 'High', 'Low', 'Close', 'Skip dates'])
    data = df_filter.values

    #Scale input
    scaler = MinMaxScaler()
    #data = scaler.fit_transform(data)

    #Create samples
    x_samples = []
    y_samples = []
    time_window = 30
    for i in range (time_window, len(data)-time_window):
      x_data = np.array(data[i-time_window : i])
      scaled_x = scaler.fit_transform(x_data)
      x_samples.append(scaled_x)
      y_samples.append([1. if data[i+30,3] > data[i, 3] else 0.])

    
    x_samples, y_samples = np.array(x_samples), np.array(y_samples)
    
    #Transform int tensors
    self.x_data = torch.from_numpy(x_samples)
    self.y_data = torch.from_numpy(y_samples)
    self.n_data = y_samples.shape[0]

    #Set transform
    self.transform = transform

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]
    if self.transform:
            sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_data

## Model

In [184]:
class LSTMPrice(nn.Module):

  def __init__(self, num_classes, input_size, hidden_size, num_layers):
    super(LSTMPrice, self).__init__()
    self.num_classes = num_classes
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.fc_1 = nn.Linear(hidden_size, 10)
    self.fc_2 = nn.Linear(10, num_classes)

  def forward(self, x):
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #hidden state
    c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    hn = hn[self.num_layers-1].view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = F.relu(hn)
    out = F.relu(self.fc_1(out)) #first Dense
    out = self.fc_2(out) #Final Output

    #Output with sigmoid
    out = torch.sigmoid(out)

    return out      


## Preparing for trainning

In [185]:
dataset = PriceDataset()

In [186]:
#Hyperparameters
batch_size=4
learning_rate=0.001
input_size=dataset[0][0].shape[1]
hidden_size=10
num_epochs=2
num_layers=2
num_classes=dataset[0][1].shape[0]

In [187]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

In [188]:
model = LSTMPrice(num_classes, input_size, hidden_size, num_layers)

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

example_input = dataset[0][0].view(1,30,5)
print(example_input)
#model(example_input.float())

tensor([[[1.0000, 1.0000, 1.0000, 1.0000, 0.0000],
         [0.5801, 0.5801, 0.5801, 0.5801, 0.0000],
         [0.5078, 0.5078, 0.5078, 0.5078, 0.0000],
         [0.4832, 0.4832, 0.4832, 0.4832, 0.0000],
         [0.5155, 0.5155, 0.5155, 0.5155, 0.0000],
         [0.6615, 0.6615, 0.6615, 0.6615, 1.0000],
         [0.4832, 0.4832, 0.4832, 0.4832, 0.0000],
         [0.4935, 0.4935, 0.4935, 0.4935, 0.0000],
         [0.2894, 0.2894, 0.2894, 0.2894, 0.0000],
         [0.3863, 0.3863, 0.3863, 0.3863, 0.0000],
         [0.3062, 0.3062, 0.3062, 0.3062, 1.0000],
         [0.3385, 0.3385, 0.3385, 0.3385, 0.0000],
         [0.2894, 0.2894, 0.2894, 0.2894, 0.0000],
         [0.2739, 0.2739, 0.2739, 0.2739, 0.0000],
         [0.1279, 0.1279, 0.1279, 0.1279, 0.0000],
         [0.0801, 0.0801, 0.0801, 0.0801, 1.0000],
         [0.0801, 0.0801, 0.0801, 0.0801, 0.0000],
         [0.1602, 0.1602, 0.1602, 0.1602, 0.0000],
         [0.1124, 0.1124, 0.1124, 0.1124, 0.0000],
         [0.1124, 0.1124, 0.112

In [189]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        inputs = inputs.float()
        labels = labels.view(batch_size, 1).float()
        
        # Forward pass
        outputs = model(inputs).float()
        print(outputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss}')
            print(outputs)

tensor([[0.4531],
        [0.4529],
        [0.4526],
        [0.4524]], grad_fn=<SigmoidBackward>)
tensor([[0.4512],
        [0.4528],
        [0.4527],
        [0.4516]], grad_fn=<SigmoidBackward>)
tensor([[0.4525],
        [0.4517],
        [0.4535],
        [   nan]], grad_fn=<SigmoidBackward>)


RuntimeError: ignored

In [ ]:
torch.empty(3).random_(2).type()